In [2]:
import fitz

In [11]:
doc = fitz.open("C:/Users/pawvaibh/Downloads/f1040filled.pdf")

In [ ]:
for page in doc:
    text = page.get_text()
    print(text)

In [ ]:
output = page.get_text(“blocks”)

In [6]:

def extract_text_with_blocks(pdf_path):
    # Open the PDF
    pdf_document = fitz.open(pdf_path)

    # Initialize an empty list to store extracted text
    extracted_text = []

    # Iterate through each page in the PDF
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)

        # Extract text with block-level segmentation
        blocks = page.get_text("blocks")

        # Iterate through each block of text
        for block in blocks:
            # Extract text and bounding box coordinates from the block
            text = block[4]  # Index 4 contains the text content of the block
            bbox = block[:4]  # Indices 0 to 3 contain the bounding box coordinates (x0, y0, x1, y1)
            block_no = block[5]

            # Append extracted text to the list
            extracted_text.append((text, bbox, block_no))

    # Close the PDF
    pdf_document.close()

    return extracted_text

extracted_text = extract_text_with_blocks("C:/Users/pawvaibh/Downloads/f1040filled.pdf")
for text, bbox, block_no in extracted_text:
    print("Text:", text)
    print("Bounding box:", bbox)
    print("Block No:", block_no)

Text: Form

Bounding box: (34.9340934753418, 38.58100128173828, 43.095977783203125, 54.91900634765625)
Block No: 0
Text: 1040 
2023
U.S. Individual Income Tax Return 

Bounding box: (43.20000076293945, 30.807987213134766, 350.0, 59.681007385253906)
Block No: 1
Text: Department of the Treasury—Internal Revenue Service 

Bounding box: (97.5999984741211, 37.621986389160156, 245.3979949951172, 44.617984771728516)
Block No: 2
Text: OMB No. 1545-0074
IRS Use Only—Do not write or staple in this space. 

Bounding box: (356.30999755859375, 49.83502197265625, 570.2979736328125, 57.99702453613281)
Block No: 3
Text: For the year Jan. 1–Dec. 31, 2023, or other tax year beginning 
, 2023, ending 
, 20 
See separate instructions.

Bounding box: (40.0, 63.33502197265625, 565.0480346679688, 73.49300384521484)
Block No: 4
Text: Your first name and middle initial 
Last name 
Your social security number 

Bounding box: (40.0, 79.57398986816406, 567.5220336914062, 87.92498779296875)
Block No: 5
Text: If jo

In [7]:


def group_text_blocks(blocks):
    # Initialize an empty list to store text clusters
    clusters = []

    # Sort blocks based on their y-coordinates (top to bottom)
    sorted_blocks = sorted(blocks, key=lambda block: block[3])

    # Initialize the current cluster with the first block
    current_cluster = [sorted_blocks[0]]

    # Iterate through each block starting from the second block
    for block in sorted_blocks[1:]:
        # If the y-coordinate of the current block is within a threshold of the previous block, add it to the current cluster
        if block[3] - current_cluster[-1][3] < 10:  # Adjust the threshold as needed
            current_cluster.append(block)
        else:
            # Start a new cluster with the current block
            clusters.append(current_cluster)
            current_cluster = [block]

    # Add the last cluster to the list
    clusters.append(current_cluster)

    return clusters

def merge_blocks_within_cluster(cluster):
    # Sort blocks within the cluster based on their x-coordinates (left to right)
    sorted_cluster = sorted(cluster, key=lambda block: block[0])

    # Initialize an empty string to store the merged text
    merged_text = ''

    # Iterate through each block in the cluster
    for block in sorted_cluster:
        # Append the text content of the block to the merged text
        merged_text += block[4] + ' '  # Add a space between text blocks

    return merged_text.strip()  # Remove trailing whitespace

def merge_adjacent_entries(clusters):
    # Initialize an empty list to store merged entries
    merged_entries = []

    # Iterate through each cluster
    for cluster in clusters:
        # Merge blocks within the cluster to form complete entries
        merged_entry = merge_blocks_within_cluster(cluster)
        # Append the merged entry to the list of merged entries
        merged_entries.append(merged_entry)

    return merged_entries

def extract_text_with_merged_entries(pdf_path):
    # Open the PDF
    pdf_document = fitz.open(pdf_path)

    # Initialize an empty list to store extracted text
    extracted_text = []

    # Iterate through each page in the PDF
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)

        # Extract text with block-level segmentation
        blocks = page.get_text("blocks")

        # Group text blocks into clusters based on their positions
        clusters = group_text_blocks(blocks)

        # Merge blocks within each cluster to form complete entries
        merged_entries = merge_adjacent_entries(clusters)

        # Append merged entries to the list of extracted text
        extracted_text.extend(merged_entries)

    # Close the PDF
    pdf_document.close()

    return extracted_text

# Example usage:
extracted_text = extract_text_with_merged_entries("C:/Users/pawvaibh/Downloads/f1040filled.pdf")
for entry in extracted_text:
    print("Entry:", entry)


Entry: Department of the Treasury—Internal Revenue Service
Entry: Form
 1040 
2023
U.S. Individual Income Tax Return 
 OMB No. 1545-0074
IRS Use Only—Do not write or staple in this space.
Entry: For the year Jan. 1–Dec. 31, 2023, or other tax year beginning 
, 2023, ending 
, 20 
See separate instructions.
 Apr. 1
Mar. 31
24
Entry: Your first name and middle initial 
Last name 
Your social security number
Entry: Vaibhav
Pawar
9
9
9
9
9
9
9
9
Entry: If joint return, spouse’s first name and middle initial
Last name 
Spouse’s social security number
Entry: Home address (number and street). If you have a P.O. box, see instructions. 
Apt. no.
Entry: 200, Baker Street
07
Entry: City, town, or post office. If you have a foreign address, also complete spaces below. 
State
ZIP code
Entry: Las Vegas
Nevada
89044
Entry: Foreign country name                                        
Foreign province/state/county                        Foreign postal code  
 Presidential Election Campaign
Check here i